In [1]:
import os 
import pandas as pd
import shutil
import time
import numpy as np


#Database
import psycopg2

### Goal: Transfer all data on sales_data folder into Postgresqldatabase

In [2]:
# get files on the source file
def get_file(data_path):
    files = [file for file in os.listdir(data_path) if file.endswith('.csv')]
    return files


# transfer files into specified directory
def transfer_files(main_path, data_dir):
    # Create directory for the datasets
    try:
        mkdir = f'mkdir {data_dir}'
        os.system(mkdir)
    except:
        print("already exist")
        
    files = get_file(data_dir)
    
    for file in files:
        try:
            mv_file = shutil.move(main_path+'/'+file, data_dir)
        except:
            pass
        
    return files


# create dataframe
def create_df(files, data_dir):
    df = {}
    data_path = os.getcwd()+'/'+data_dir+'/'
    for file in files:
        df[file] = pd.read_csv(data_path+file)

    return df

# Clean and Transform data
def transform_data(data):
    #columns to convert dtypes
    columns_num = ['Order ID', 'Quantity Ordered','Price Each']
    columns_date = ['Order Date']
    
    data.drop_duplicates(inplace=True, ignore_index = True)
    
    for col in columns_num:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    
    for col in columns_date:
        data[col] = pd.to_datetime(data[col], format='%m/%d/%y %H:%M', errors='coerce')
    
            
    data.dropna(inplace=True, ignore_index = True)
    
    return data

In [4]:
# Main

if __name__ == "__main__":
    main_path = '/Users/chano/Programming files'
    data_dir = '2019_sales_data'

    files = transfer_files(main_path, data_dir)
    df = create_df(files,data_dir)
    
    columns_int = ['Order ID', 'Quantity Ordered']
    
    for file in files:
        errors = []
        data = df[file]

        # removing the .csv on the filename
        table_name = '{0}'.format(file.split('.')[0]).replace(' ','_')

        
        clean_data = transform_data(data)
        for col in columns_int:
            clean_data[col] = clean_data[col].apply(np.int64)
        
        # Clean columns
        data.columns = [col.replace(' ','_') for col in clean_data.columns]  
        
        # replacement dictionary, it maps the pandas dtypes to sql dtypes
        replacements = {
        'object': 'varchar',
        'int64' : 'int',
        'float64': 'float',
        'datetime64[ns]':'timestamp'
        }

        # columns table constraint
        col_sql = " , ".join('{} {}'.format(n ,d) for (n, d) in zip(data.columns, data.dtypes.replace(replacements)))
        
        # Connect to database
        conn = psycopg2.connect(
           database="sales_data_2019", 
            user='postgres', 
            password='XXXXXXX', 
            host='XXXXXXX', 
            port= 'XXXXXX'
            )
        cur = conn.cursor()
        
        # Adding tables to database
        cur.execute(f"""DROP TABLE IF EXISTS {table_name}""")
        cur.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} ({col_sql});
        """)
        
        # Saving dataframe to csv
        data.to_csv(file, header=data.columns, index=False)
        
        #Open csv and store into memory
        my_file = open(file)
        print(f"{table_name} was successfully created!")
        
        # upload csv to database 
        # Need to review
        
        sql_statement = f"""
            COPY {table_name} FROM STDIN WITH
                CSV
                HEADER
                DELIMITER AS ','
        """
        
        cur.copy_expert(sql=sql_statement, file=my_file)
        print("Values successfully added!")
        
        conn.commit()
        conn.close()

Sales_April_2019 was successfully created!
Values successfully added!
Sales_August_2019 was successfully created!
Values successfully added!
Sales_December_2019 was successfully created!
Values successfully added!
Sales_February_2019 was successfully created!
Values successfully added!
Sales_January_2019 was successfully created!
Values successfully added!
Sales_July_2019 was successfully created!
Values successfully added!
Sales_June_2019 was successfully created!
Values successfully added!
Sales_March_2019 was successfully created!
Values successfully added!
Sales_May_2019 was successfully created!
Values successfully added!
Sales_November_2019 was successfully created!
Values successfully added!
Sales_October_2019 was successfully created!
Values successfully added!
Sales_September_2019 was successfully created!
Values successfully added!
